# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 23 2023 8:30AM,257981,10,0086420709,ISDIN Corporation,Released
1,Feb 23 2023 8:30AM,257981,10,0086420720,ISDIN Corporation,Released
2,Feb 23 2023 8:30AM,257981,10,0086420721,ISDIN Corporation,Released
3,Feb 23 2023 8:30AM,257981,10,0086420726,ISDIN Corporation,Released
4,Feb 23 2023 8:30AM,257993,10,0086420740,ISDIN Corporation,Released
5,Feb 23 2023 8:30AM,257993,10,0086420763,ISDIN Corporation,Released
6,Feb 23 2023 8:30AM,257993,10,0086420764,ISDIN Corporation,Released
7,Feb 23 2023 8:30AM,257993,10,0086420765,ISDIN Corporation,Released
8,Feb 23 2023 8:30AM,257993,10,0086420995,ISDIN Corporation,Released
9,Feb 23 2023 8:30AM,257993,10,0086420996,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
23,257993,Released,12
24,257996,Released,1
25,257997,Released,1
26,257999,Released,3
27,258000,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257993,NaN,NaN,12.0
257996,NaN,NaN,1.0
257997,NaN,NaN,1.0
257999,NaN,NaN,3.0
258000,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257914,10.0,3.0,6.0
257916,0.0,0.0,1.0
257934,0.0,0.0,2.0
257935,1.0,0.0,16.0
257945,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257914,10,3,6
257916,0,0,1
257934,0,0,2
257935,1,0,16
257945,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257914,10,3,6
1,257916,0,0,1
2,257934,0,0,2
3,257935,1,0,16
4,257945,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257914,10,3,6
1,257916,,,1
2,257934,,,2
3,257935,1,,16
4,257945,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 23 2023 8:30AM,257981,10,ISDIN Corporation
4,Feb 23 2023 8:30AM,257993,10,ISDIN Corporation
16,Feb 23 2023 8:30AM,257990,10,ISDIN Corporation
34,Feb 23 2023 8:30AM,257978,10,"Methapharm, Inc."
39,Feb 23 2023 8:30AM,257988,10,"Citieffe, Inc."
40,Feb 23 2023 8:30AM,257988,10,Citieffe Archive
41,Feb 23 2023 8:30AM,257978,10,Methapharm-G
42,Feb 23 2023 8:30AM,257999,10,"StellaLife, Inc."
45,Feb 22 2023 5:26PM,258000,21,"NBTY Global, Inc."
46,Feb 22 2023 4:51PM,257997,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 23 2023 8:30AM,257981,10,ISDIN Corporation,,,9
1,Feb 23 2023 8:30AM,257993,10,ISDIN Corporation,,,12
2,Feb 23 2023 8:30AM,257990,10,ISDIN Corporation,,,13
3,Feb 23 2023 8:30AM,257978,10,"Methapharm, Inc.",5,,1
4,Feb 23 2023 8:30AM,257978,10,Methapharm-G,5,,1
5,Feb 23 2023 8:30AM,257988,10,"Citieffe, Inc.",,,2
6,Feb 23 2023 8:30AM,257988,10,Citieffe Archive,,,2
7,Feb 23 2023 8:30AM,257999,10,"StellaLife, Inc.",,,3
8,Feb 22 2023 5:26PM,258000,21,"NBTY Global, Inc.",,,1
9,Feb 22 2023 4:51PM,257997,22,"NBTY Global, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 8:30AM,257981,10,ISDIN Corporation,9,,
1,Feb 23 2023 8:30AM,257993,10,ISDIN Corporation,12,,
2,Feb 23 2023 8:30AM,257990,10,ISDIN Corporation,13,,
3,Feb 23 2023 8:30AM,257978,10,"Methapharm, Inc.",1,,5
4,Feb 23 2023 8:30AM,257978,10,Methapharm-G,1,,5
5,Feb 23 2023 8:30AM,257988,10,"Citieffe, Inc.",2,,
6,Feb 23 2023 8:30AM,257988,10,Citieffe Archive,2,,
7,Feb 23 2023 8:30AM,257999,10,"StellaLife, Inc.",3,,
8,Feb 22 2023 5:26PM,258000,21,"NBTY Global, Inc.",1,,
9,Feb 22 2023 4:51PM,257997,22,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 8:30AM,257981,10,ISDIN Corporation,9,,
1,Feb 23 2023 8:30AM,257993,10,ISDIN Corporation,12,,
2,Feb 23 2023 8:30AM,257990,10,ISDIN Corporation,13,,
3,Feb 23 2023 8:30AM,257978,10,"Methapharm, Inc.",1,,5
4,Feb 23 2023 8:30AM,257978,10,Methapharm-G,1,,5


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 8:30AM,257981,10,ISDIN Corporation,9.0,NaN,NaN
1,Feb 23 2023 8:30AM,257993,10,ISDIN Corporation,12.0,NaN,NaN
2,Feb 23 2023 8:30AM,257990,10,ISDIN Corporation,13.0,NaN,NaN
3,Feb 23 2023 8:30AM,257978,10,"Methapharm, Inc.",1.0,NaN,5.0
4,Feb 23 2023 8:30AM,257978,10,Methapharm-G,1.0,NaN,5.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 23 2023 8:30AM,257981,10,ISDIN Corporation,9.0,0.0,0.0
1,Feb 23 2023 8:30AM,257993,10,ISDIN Corporation,12.0,0.0,0.0
2,Feb 23 2023 8:30AM,257990,10,ISDIN Corporation,13.0,0.0,0.0
3,Feb 23 2023 8:30AM,257978,10,"Methapharm, Inc.",1.0,0.0,5.0
4,Feb 23 2023 8:30AM,257978,10,Methapharm-G,1.0,0.0,5.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4643427,219.0,3.0,34.0
21,773891,2.0,1.0,0.0
22,1031901,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4643427,219.0,3.0,34.0
1,21,773891,2.0,1.0,0.0
2,22,1031901,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,219.0,3.0,34.0
1,21,2.0,1.0,0.0
2,22,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,219.0
1,21,Released,2.0
2,22,Released,4.0
3,10,Executing,3.0
4,21,Executing,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,21,22
Status,,,
Completed,34.0,0.0,0.0
Executing,3.0,1.0,0.0
Released,219.0,2.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,21,22
0,Completed,34.0,0.0,0.0
1,Executing,3.0,1.0,0.0
2,Released,219.0,2.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,21,22
0,Completed,34.0,0.0,0.0
1,Executing,3.0,1.0,0.0
2,Released,219.0,2.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()